In [1]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re
import pickle
import time

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from tqdm.auto import tqdm, trange

from nltk.corpus import stopwords

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mikha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mikha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mikha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_train = pd.read_json("news_train.json", lines=True)
data_train.head()

,category,headline,authors,link,short_description,date
0,PARENTING,A Matter of Basic Fairness and Opportunity,"Bruce Lesley, Contributor\nChild advocate, who...",https://www.huffingtonpost.com/entry/a-matter-...,A 21st century education system must be built ...,2013-04-05
1,WELLNESS,Olympic Ski Jumper Jessica Jerome On Mental Fo...,,https://www.huffingtonpost.com/entry/jessica-j...,"What are your goals for Sochi? Of course, I wa...",2014-02-11
2,POLITICS,Opioid Epidemic Goes Unmentioned In First Pres...,Jason Cherkis,https://www.huffingtonpost.com/entry/opioid-ep...,But the issue is a big concern in key swing st...,2016-09-27
3,GREEN,25 Animal Pals Who Know What Friendship Is All...,Kate Bratskeir,https://www.huffingtonpost.com/entry/animals-r...,,2014-07-06
4,POLITICS,Former Virginia Lawmaker Joe Morrissey Distrib...,Sam Levine,https://www.huffingtonpost.com/entry/joe-morri...,,2015-05-23


In [3]:
data_train.count()

category             189527
headline             189527
authors              189527
link                 189527
short_description    189527
date                 189527
dtype: int64

In [4]:
data_test = pd.read_json("news_test.json", lines=True)
data_test.head()
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   category           20000 non-null  object        
 1   headline           20000 non-null  object        
 2   authors            20000 non-null  object        
 3   link               20000 non-null  object        
 4   short_description  20000 non-null  object        
 5   date               20000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 937.6+ KB


In [5]:
data_test.count()
lem = WordNetLemmatizer()

In [6]:
data_train['data'] =  data_train['headline'] + ' ' + data_train['authors'] + ' ' + data_train['link'] + ' ' + data_train['short_description']
data_train.head()

,category,headline,authors,link,short_description,date,data
0,PARENTING,A Matter of Basic Fairness and Opportunity,"Bruce Lesley, Contributor\nChild advocate, who...",https://www.huffingtonpost.com/entry/a-matter-...,A 21st century education system must be built ...,2013-04-05,A Matter of Basic Fairness and Opportunity Bru...
1,WELLNESS,Olympic Ski Jumper Jessica Jerome On Mental Fo...,,https://www.huffingtonpost.com/entry/jessica-j...,"What are your goals for Sochi? Of course, I wa...",2014-02-11,Olympic Ski Jumper Jessica Jerome On Mental Fo...
2,POLITICS,Opioid Epidemic Goes Unmentioned In First Pres...,Jason Cherkis,https://www.huffingtonpost.com/entry/opioid-ep...,But the issue is a big concern in key swing st...,2016-09-27,Opioid Epidemic Goes Unmentioned In First Pres...
3,GREEN,25 Animal Pals Who Know What Friendship Is All...,Kate Bratskeir,https://www.huffingtonpost.com/entry/animals-r...,,2014-07-06,25 Animal Pals Who Know What Friendship Is All...
4,POLITICS,Former Virginia Lawmaker Joe Morrissey Distrib...,Sam Levine,https://www.huffingtonpost.com/entry/joe-morri...,,2015-05-23,Former Virginia Lawmaker Joe Morrissey Distrib...


In [7]:
data_test['data'] =  data_test['headline'] + ' ' + data_test['authors'] + ' ' + data_test['link'] + ' ' + data_test['short_description']
data_test.head()

,category,headline,authors,link,short_description,date,data
0,PARENTS,Dear Family Whisperer: Don't Get a Dog 'For' Y...,"Melinda Blau, ContributorAuthor, 'Family Whisp...",https://www.huffingtonpost.com/entry/dear-fami...,"Your ""mistake"" isn't getting a dog. It's ""givi...",2015-04-23,Dear Family Whisperer: Don't Get a Dog 'For' Y...
1,GREEN,Family Fights To Ban USDA Wildlife Trap That K...,Hilary Hanson,https://www.huffingtonpost.com/entry/mansfield...,The cyanide-spraying device was placed to kill...,2017-04-01,Family Fights To Ban USDA Wildlife Trap That K...
2,POLITICS,More Farmers Around The World Water Crops With...,Alexander C. Kaufman,https://www.huffingtonpost.com/entry/climate-w...,Climate change could make the problem much worse.,2017-07-11,More Farmers Around The World Water Crops With...
3,ENTERTAINMENT,Taylor Swift Goes Back To Her Country Roots To...,Julia Brucculieri,https://www.huffingtonpost.com/entry/taylor-sw...,"If you love the old Taylor, you'll probably lo...",2016-11-01,Taylor Swift Goes Back To Her Country Roots To...
4,WORLDPOST,Afghanistan 2015: The Future of the Taliban,"Joseph V. Micallef, ContributorBest Selling Mi...",https://www.huffingtonpost.com/entry/afghanist...,If the United States and NATO forces withdraw ...,2015-12-13,Afghanistan 2015: The Future of the Taliban Jo...


In [8]:

def text_conversion(text):
    lem = WordNetLemmatizer()
    text = text.lower()
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(text))
    text = re.sub(r'\^[a-zA-Z]{1,2}\s+', ' ', str(text))
    text = re.sub(r'\s+', ' ', str(text), flags=re.I)
    return ' '.join([ lem.lemmatize(word) for word in text.split()]) 

In [9]:
data_train['data'] = data_train['data'].apply(text_conversion)
data_test['data'] = data_test['data'].apply(text_conversion)

In [10]:
X_test = data_test['data']
X_train = data_train['data']

In [11]:
Y_test = data_test['category']
Y_train = data_train['category']

In [12]:
tf_vectorizer = TfidfVectorizer(max_df=0.9, ngram_range=(1, 2), use_idf=True, stop_words=stopwords.words('english'))
X_train = tf_vectorizer.fit_transform(X_train)
X_test = tf_vectorizer.transform(X_test)

In [13]:
linear_svc = LinearSVC(fit_intercept= True, loss= 'squared_hinge', max_iter = 900, multi_class = 'ovr', random_state= 42)
linear_svc.fit(X_train, Y_train)

LinearSVC(max_iter=900, random_state=42)

In [14]:
predicted = linear_svc.predict(X_test)
print(classification_report(Y_test,predicted)) 
print(f1_score(Y_test, predicted, average='macro')) 

                precision    recall  f1-score   support

          ARTS       0.78      0.72      0.75       155
ARTS & CULTURE       0.76      0.65      0.70       110
  BLACK VOICES       0.76      0.63      0.69       438
      BUSINESS       0.70      0.71      0.71       561
       COLLEGE       0.64      0.52      0.57       121
        COMEDY       0.77      0.71      0.73       518
         CRIME       0.63      0.63      0.63       356
CULTURE & ARTS       0.88      0.50      0.64       105
       DIVORCE       0.90      0.82      0.86       337
     EDUCATION       0.62      0.54      0.58        99
 ENTERTAINMENT       0.81      0.84      0.82      1642
   ENVIRONMENT       0.78      0.46      0.58       156
         FIFTY       0.84      0.78      0.81       135
  FOOD & DRINK       0.87      0.90      0.88       618
     GOOD NEWS       0.67      0.59      0.63       127
         GREEN       0.63      0.63      0.63       243
HEALTHY LIVING       0.71      0.71      0.71  